### 特徵應用
當我們取得特徵之後，就代表我們已經有能力去識別關鍵點的特殊性

在這之後可以接到許多電腦視覺的任務
- 配對：判斷兩張圖片上相同物體的位置
- 辨識：判斷兩張圖片上是否有相同物體
- 全景圖：尋找兩張圖片的相同視角，再經過轉換合成全景圖
- ...
廣泛的說，SIFT 只是其中一種抽取特徵的方式，這邊會延續上一章節以 SIFT 為例介紹配對的應用。

### Feature Matching

配對會從兩張圖片中的關鍵點中，透過計算其特徵空間上的距離，若小於一個設定的閥值就視為是相同的特徵

在 SIFT 特徵的配對任務中，通常會使用 L2 norm 的方式計算。兩個 128 維向量(敘述子)可以得到一個距離

actually it's similiarity

簡單暴力的配對方法是逐一針對 query image 的關鍵點

對每個 train image 的關鍵點計算 L2 距離

- 取得距離最小的配對
- 取得 k 個最適合的配對(針對每個query都找k個適合的)

這邊為了確保配對的合適性，可以先在計算時取得 k 個配對，在根據距離(ratio test)去過濾不適合的配對

#### ratio test

我們可以尋找 k=2 個最好的 match 方式，透過 ratio test 的方式來過濾一些不適當的配對，因為有時候 query 的關鍵點並不會出現在 train image( 0.7~0.8 is suggested)

# 範例

透過 SIFT 特徵實作 Brute-Force Matching

In [1]:
import cv2
import numpy as np

# 以灰階方式讀入圖片
img_query = cv2.imread('box.png', 0)
img_train = cv2.imread('box_in_scene.png', 0)

# 建立 SIFT 物件
sift = cv2.SIFT_create()

# 偵測並計算 SIFT 特徵 (keypoints 關鍵點, descriptor 128 維敘述子)
kp_query, des_query = sift.detectAndCompute(img_query, None) #None是mask參數 可以針對部分圖片做處理
kp_train, des_train = sift.detectAndCompute(img_train, None)

## 基於 SIFT 特徵的暴力比對

- D.Lowe ratio test
- knn 比對

In [2]:
# 建立 Brute-Force Matching 物件
bf = cv2.BFMatcher(cv2.NORM_L2)

# 以 knn 方式暴力比對特徵
matches = bf.knnMatch(des_query, des_train, k=2)

# 透過 D.Lowe ratio test 排除不適合的配對
candidate = []
for m, n in matches:
    if m.distance < 0.75*n.distance: #m is closest and n is next closest
        candidate.append([m])

# 顯示配對結果
img_show = cv2.drawMatchesKnn(img_query, kp_query, img_train, kp_train, candidate, None, flags=2) #flag=2沒有配對成功不會被標記

# 顯示圖片
while True:
    cv2.imshow('matches', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

### related to ML

SIFT 可以抽取特徵，ML也可以抽取特徵

SIFT抽取完後做ML，注意維度問題(每張圖片特徵點數量不同)

其中一種作法是做 Clustering，每一張圖片都取 n 個特徵點來固定圖片的特徵維度

缺點：

如果圖片太簡單導致部份圖片特徵太少就會失效，所以類似 MNIST 或是 CIFAR 等簡單的資料集就不太適合